# 机器学习与社会科学应用

# 第六章 无监督学习算法

# 第四节 LDA主题模型

<font face="宋体" >郭峰    
    教授、博士生导师  
上海财经大学公共经济与管理学院  
上海财经大学数实融合与智能治理实验室   
    邮箱：guofengsfi@163.com</font> 

<font face="宋体" >本节目录  
4.1.数据预处理   
4.2.训练LDA模型   
4.3.确定最优主题数   
4.4.最优主题数的主题图  
4.5.绘制主题词云图  
4.6.两篇文章的主题相似度</font>  

## 4.1. 数据预处理

In [ ]:
import pandas as pd 
import numpy as np
import re 
import os 
import time
from tqdm import tqdm
# import warnings
# warnings.filterwarnings("ignore")

os.chdir('D:/python/机器学习与社会科学应用/演示数据/06无监督学习/lda主题模型/')
df = pd.read_csv('cssci_clean_test.csv', encoding='utf8')
print("原始样本量：",df.shape)

# 保存需要的列，并更改列名
df = df[['title', 'keyword', 'abstract']]


# 将标题、摘要、内容合并
df['keyword'] = df['keyword'].fillna(";")
df['content'] = df['title'] + ';' + df['keyword'] + df['abstract']
df = df[df['content'].str.len() > 100]
print("标题+内容大于100字的数量：", len(df))

# 剔除标题和内容为空的行
df = df.dropna()
# 剔除重复行
df = df.drop_duplicates()
# 剔除内容字数少于100的行
df = df[df['content'].str.len() > 100]
print("剔除Title为空的行以及重复的行后的数量：", len(df))

df.to_csv('cssci_lda.csv', encoding='utf8', index=False)

## 4.2. 训练LDA模型

In [ ]:
import pandas as pd 
import numpy as np 
import jieba
import jieba.analyse
import re
import gensim 
import pyLDAvis
import pyLDAvis.gensim_models
import os 
import time
from collections import defaultdict
from tqdm import tqdm
from cntext.dictionary import STOPWORDS_zh
import warnings
warnings.filterwarnings("ignore")

os.chdir('D:/python/机器学习与社会科学应用/演示数据/06无监督学习/lda主题模型/')

# 分词
def cut_words(string):
    string = re.sub(r"[0-9a-z\s+]+", "",string)    
    # 加载自定义词典
    jieba.load_userdict("keywords.txt")
    # 删除单个字符的词
    cuts = [w for w in jieba.cut(string) if len(w)>1]
    # 删除停用词
    cuts = [w for w in cuts if w not in STOPWORDS_zh]   
    return cuts

# 创建词袋向量（bag of words）
def create_bow(texts_cut):
#     frequency = defaultdict(int)
#     for text in texts_cut:
#         for token in text:
#             frequency[token] += 1
    # texts_cut = [[token for token in text if frequency[token] > 0] for text in texts_cut]  #只保留词频大于1的
    dictionary = gensim.corpora.Dictionary(texts_cut)
    # print(len(dictionary))
    # print(dictionary.token2id)    
    # 过滤，每个词汇至少在30篇文档中存在，每个词至多存在于99%的文档，保留最高词频30000个词
    dictionary.filter_extremes(no_below=30, no_above=0.9, keep_tokens=None)
    # print(len(dictionary))
    # 删除词频最高的两个词
    dictionary.filter_n_most_frequent(2)
    corpus = [dictionary.doc2bow(text) for text in text_cut]
    return dictionary, corpus

# 训练并保存lda模型
def training_ldamodel(topic_num):
    lda = gensim.models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=topic_num, passes=10)
    if not os.path.exists("lda_model/" + str(topic_num)):
        os.makedirs("lda_model/" + str(topic_num))
    lda.save('lda_model/'+ str(topic_num) + '/' + 'lda_model.model')  

    
if __name__ == '__main__':
    starttime = time.time()
    # *****************【此部分需要修改】
    # 读取数据
    df = pd.read_csv('cssci_lda.csv')
    print("语料库样本Size:", df.shape)
    df = df.sample(5000) # 演示时为了减少时间，减少数据规模
    # 文档所在的列名
    content = 'content'  
    # 设定主题数目区间
    topics_range = range(5, 15)  # 【根据需要设定主题数目的范围】
    
    # 分词
    tqdm.pandas()
    df['text_cut'] = df[content].progress_apply(cut_words)
    # 如果数据量大可以保存分词结果
    df.to_csv('cssci_lda_cut.csv', encoding='utf8', index=False)
    # 创建词袋向量
    text_cut = df['text_cut'].tolist()  # 将列转化为数组形式
    dictionary, corpus = create_bow(text_cut)
    
    # 训练LDA模型    
    for i in tqdm(topics_range):
        training_ldamodel(i)
    # 保存corpus,由于dictionary会随着LDA model的保存而一并保存，corpus需要单独保存
    gensim.corpora.MmCorpus.serialize('lda_model/corpus.mm', corpus)

    endtime = time.time()
    print("总耗时：", (endtime - starttime))

## 4.3. 确定最优主题数

评价模型：  
- 通常用于评价聚类算法好坏的方法有两种，其一是使用带分类标签的测试数据集，然后使用一些算法来判断聚类结果与真实结果的差距；其二是使用无分类标签的测试数据集，通过一些指标进行衡量，通常衡量LDA主题的指标有两个：困惑度与一致性，其中一致性指标更好。
- 困惑度指标可以通俗的理解为某个文档属于某个主题的概率大小，如果概率越小，表示困惑越大，即越不可能属于这个主题。因此困惑的越小越好。
- 一致性指标可以通俗的理解为每个主题内的主题词的相似性。例如，一个主题都是关于农业的词汇，包括农村、农民、乡村、化肥等词，显然一致性比较稿；但是如果这个关于农业的主题还包括股票、货币、资本等词汇，显然这个主题的一致性就比较低。
- 因此，对于LDA主题模型评价要求困惑度越低越好，一致性越高越高。在实践中我们通常画出LDA的困惑度和一致性指标的折线图进行评估，有时还需要结合具体的主题的主题关键词进行判断是否最优。

### 困惑度

- log_perplexity()这个函数没有对主题数目做归一化，因此不同的topic数目不能直接比较.
- gensim包的作者Radim现身回答说perplexity不是一个好的评价topic质量的指标
- 参考：https://blog.csdn.net/qq_23926575/article/details/79472742
- 困惑度的公式
$$
perplexity(D)=exp{\frac{-\sum^M_{d=1}log{p(w_d)}}{\sum^M_{d=1}N_d}}
$$
其中，M是测试语料库的大小，$N_d$是第d篇文本的大小（即单词个数）。
$$
p(w_d)=\sum_z p(z)p(w|z, gamma)
$$
其中z是主题，w是文档，gamma是训练集学出来的文本-主题分布。

所以perlexity的上半部分就是生成整个文档的似然估计（表示训练集训练出的参数的生成能力）的负值，由于概率值取值范围为[0, 1]，按照对数函数的定义，分子值是一个大数，而分母是整个测试集的单词数目。也就是说模型生成能力越强，perplexity值越小。

###  一致性  

- 更好的内在评估方法
- 衡量生成的主题的连贯性
- 一个好的主题模型能够生成更连贯的主题

单词入侵  
- 想法：为主题注入一个随机词，比如农业主题{farmers, farm, food, rice, agricluture}$\rightarrow${farmers, farm, food, rice, <font color=red>cat</font>, agriculture}
- 让用户猜测哪个是入侵词
- 猜对$\rightarrow$ 话题连贯

PMI$\approx$一致性
- 一对词的PMI高$\rightarrow$词是相关的；PMI(farm, rice)较高，PMI(farmers, cat)较低；
- 如果主题中的所有词对都具有高PMI$\rightarrow$主题是连贯的
- 如果大多数主题具有高PMI$\rightarrow$良好的主题模型
- 哪里可以获得PMI的词共现统计数据
    - 主题模型可以使用相同的语料库
    - 更好的方法是使用外部语料库（例如维基百科）
    
PMI
- 计算主题中前N个词的称对PMI
$$
PMI(t) = \sum^N_{j=2}\sum^{j-1}_{i=1}log{\frac{P(w_i, w_j)}{P(w_i)P(w_j)}}
$$
- 给定主题：{farmers, farm, food, rice, agriculture}
- 连贯性 = 所有词对的总和PMI：
    - PMI(farmers, farm)+PMI(farmers, food)+...+PMI(rice, agriculture)
- 标准化PMI
$$
NPMI(t) = \sum^N_{j=2}\sum^{j-1}_{i=1} \frac{log{\frac{P(w_i, w_j)}{P(w_i)P(w_j)}}}{-logP(w_i,w_j)}
$$

In [ ]:
# 困惑度或一致性指标与主题个数折线图
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd 
import numpy as np
import time
import gensim
from gensim.models.coherencemodel import CoherenceModel
import ast
from tqdm import tqdm
import os 

os.chdir('D:/python/机器学习与社会科学应用/演示数据/06无监督学习/lda主题模型/')


#计算困惑度
def compute_perplexity(topic_num):
    ldamodel = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model')
    # corpus = gensim.corpora.MmCorpus('lda_model/corpus.mm')
    return ldamodel.log_perplexity(corpus)


#计算一致性指标
def compute_coherence(topic_num):
    ldamodel = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model')
    dictionary = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model.id2word')
    # corpus = gensim.corpora.MmCorpus('lda_model/corpus.mm')
    ldacm = CoherenceModel(model=ldamodel, texts=text_cut, dictionary=dictionary, coherence='c_v')
    return ldacm.get_coherence()

if __name__ == '__main__':
    starttime = time.time()
    df = pd.read_csv('cssci_lda_cut.csv')  # 【需要修改】
    df['text_cut'] = df['text_cut'].map(ast.literal_eval)
    text_cut = df['text_cut'].tolist()  # 将列转化为数组形式
    corpus = gensim.corpora.MmCorpus('lda_model/corpus.mm')
       
    x = range(5, 15)   # 【与前文的topics_range一致】
    # 困惑度
    y = [compute_perplexity(i) for i in tqdm(x)]
    # 一致性
    z = [compute_coherence(i) for i in tqdm(x)] 
    print("perplexity and coherence are: ", y, z)

    plt.subplot(2, 1, 1)
    plt.plot(x, y, 'ko-')
    plt.xlabel('topics_num')
    plt.ylabel('perplexity size')
    plt.rcParams['font.sans-serif']=['SimHei']
    matplotlib.rcParams['axes.unicode_minus']=False
    plt.title('topics-perplexity')

    plt.subplot(2, 1, 2)
    plt.plot(x, z, 'r.-')
    plt.xlabel('topics_num')
    plt.ylabel('coherence size')
    plt.title('topics-coherence')
    plt.show()
    endtime = time.time()
    print("总耗时：", (endtime - starttime))
    
# 从下面的困惑度与一致性折线图可以看出，困惑度的折线图从5-11缓慢上升，主题数为12时有所下降，此后开始大幅上升
# 因此根据困惑图，我们将主题数定在12及以下比较合适
# 再看一致性图，在主题数为7-9时，一致性指标达到较高的之，此后大幅下降，并从主题数12时后又大幅上升
# 通过困惑度图和一致性图，我们初步将主题数定为7-9，后面我们将根据具体的主题词分布正式确定主题数。

## 4.4. 最优主题数的主题图

上文中，我们初步确定主题数为7-9，这里，我们查看每个主题的交叉情况和关键词的分布情况，最终确定主题数。

In [ ]:
import pyLDAvis 
from pyLDAvis import gensim_models
import gensim
from gensim import corpora
import os 

os.chdir('D:/python/机器学习与社会科学应用/演示数据/06无监督学习/lda主题模型/')

# 加载保存的LDA模型
topic_num = 8 # 【依次输入7、8、9，观察每个主题模型的主题分布】
lda_model = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model')
dictionary = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model.id2word')
corpus = corpora.MmCorpus('lda_model/corpus.mm')

# 输出模型
print(lda_model.print_topics(num_topics=topic_num, num_words=3))  # 输出10个主题，每个主题4个词汇
print(lda_model.print_topic(0, topn=5)) # 输出第0个主题，排名前十的关键词
    
# 主题可视化
pyLDAvis.enable_notebook(local=False)
plot = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
# 保存主题可视化为网页，但是由于网页依赖的第三方插件存在问题，打开前使用TXT打开并将文本中的“cdn”替换为“fastly”，然后关闭再用浏览器打开
# pyLDAvis.save_html(plot, 'lda_model/'+ str(topic_num) + '/'+'LDAvis.html')
# pyLDAvis.display(plot, local=True)  # 【有时需要单独运行才有效】

In [ ]:
pyLDAvis.display(plot, local=True)  # 【有时需要单独运行才有效】
# 图示说明
# 左图：每一个圆圈代表一个主题，圆的大小表示主题再在文档中所占的比例，如第一个主题的圆面积最大，表示该主题在所有文档中所占的比例最大，圆面积随着主题顺序的递增而逐渐减少，表示后面的主题在文档的分布越来越小
# 圆与圆之间的距离表示两者的相似度，距离越近，相似度越高。因此，圆圈越分散，表示主题的分类越清晰，模型越好。
# 右图柱状图，灰色代表总的词频，红色代表主题内的关键词词频
# 相关性lambda可以右上角蓝色条进行调整，含义类似于TF-IDF
# 通过对比，发现主题数为7和8时，分类效果较好，最终的分类还可以根据每个主题的关键词进行判断
# 通过观察，我们发现主题数为8时，主题的关键词的一致性较好，因此最终我们选择主题数为8
# 在实践中，不一定需要确定最优主题，可以根据需要选择与用户需要的主题相似的主题数就可以了。

## 4.5. 绘制主题词云图

In [ ]:
#LDA主题模型的关键词词云
import codecs
import numpy as np
import pandas as pd
import gensim 
# from wordcloud import WordCloud, ImageColorGenerator
# from PIL import Image
from pyecharts.charts import WordCloud
from pyecharts import options as opts
import os

os.chdir('D:/python/机器学习与社会科学应用/演示数据/06无监督学习/lda主题模型/')

topic_num = 8 #【设定主题数目】
topicn = 2 #【第几个主题】

lda = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model')
#topic_word = lda.print_topics(num_topics=1, num_words=10)
#print(topic_word)
topic_word = lda.print_topic(topicn,topn=100)
print(topic_word)
topic_word = topic_word.split(' + ')
word_score_list = []
for tw in topic_word:
    temp1 = tw.split('*')
     #print(temp1)
    temp1[1] = eval(temp1[1])
    word_score_list.append(temp1)
#print(word_score_list)
print(word_score_list[:][0])
word=[x[1] for x in word_score_list]
count=[x[0] for x in word_score_list]
print(word[0:10])
print(count[0:10])

data=[]
for i in range(len(word_score_list)):
    temp=[(word[i],count[i])]
    data=data+temp
    
# 创建实例对象
c = WordCloud()
c.add(series_name="",data_pair=data)
# 设置标题
c.set_global_opts(title_opts=opts.TitleOpts("主题词云"))
# 展示图片
c.render_notebook()  

## 4.6. 计算两篇文档的相似度

In [ ]:
import pandas as pd 
import numpy as np 
import gensim 
import jieba 
import scipy.stats
import time
import os 
import ast

os.chdir('D:/python/机器学习与社会科学应用/演示数据/06无监督学习/lda主题模型/')

def lda_similarity(text1, text2):
    # 文档1
#     text1 = text1.split()
    print(text1)
    corpus1 = dictionary.doc2bow(text1)
    lda_text1 = lda_model[corpus1]
    # 输出新文档的主题分布
    print(lda_text1)
    list_text1 = [i[1] for i in lda_text1]
    print('list_text1: ', list_text1)
    
    # 文档2
#     text2 = text2.split()
    print(text2)
    corpus2 = dictionary.doc2bow(text2)
    lda_text2 = lda_model[corpus2]
    print(lda_text2)
    list_text2 = [i[1] for i in lda_text2]
    print('list_text2: ', list_text2)
    
    # 相似度计算
    try:
        # 欧式距离相似度；dot()返回的时两个数组的点积；np.linalg.norm求欧式距离；值越大表示越相似
        # sim = np.dot(list_text1, list_text2) / (np.linalg.norm(list_text1) * np.linalg.norm(list_text2))
        # KL divergence，也称KL散度，衡量两个概率分布的相似性；散度越小，即距离越小，表示两个概率分布越相似
        sim = scipy.stats.entropy(list_text1, list_text2)
    except ValueError:
        sim = 0
        
if __name__ == '__main__':
    starttime = time.time()
    topic_num = 8 #【设定主题数目】
    # 加载模型
    lda_model = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model')
    dictionary = gensim.models.ldamodel.LdaModel.load('lda_model/'+ str(topic_num) + '/' + 'lda_model.model.id2word')
    # 读取数据
    df = pd.read_csv('cssci_lda_cut.csv')
    df['text_cut'] = df['text_cut'].map(ast.literal_eval)
    df = df.sample(200).reset_index()
    text1 = df['text_cut'][20] #【第20个文档】
    text2 = df['text_cut'][50] #【第50个文档】
    sim = lda_similarity(text1, text2)
    endtime = time.time()
    print("总耗时：",(endtime - starttime))